# Testing Layers

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from fastai.vision import *
from octconv import TensorTuple, tuplify, tuplify_bn, OctConv
from octxresnet import conv_layer, octxresnet50

## Test Tensor Tuples

In [2]:
bs, ch, h, w = 16, 128, 224, 224

In [3]:
rand1 = torch.rand((bs, ch, h, w))
rand2 = torch.rand((bs, ch, h, w))
rand3 = torch.rand((bs, ch, h, w))
rand4 = torch.rand((bs, ch, h, w))

rand1.shape

torch.Size([16, 128, 224, 224])

In [4]:
tt1 = TensorTuple(rand1, rand2)
tt2 = TensorTuple(rand3, rand4)

In [5]:
tt1.shape

(torch.Size([16, 128, 224, 224]), torch.Size([16, 128, 224, 224]))

In [6]:
out = (tt1 + tt2)
out.shape

(torch.Size([16, 128, 224, 224]), torch.Size([16, 128, 224, 224]))

In [7]:
out.lf[0, 0, :5, :5], out.hf[0, 0, :5, :5]

(tensor([[1.4519, 1.3519, 0.9868, 0.8336, 0.9742],
         [0.2824, 0.2967, 1.1428, 1.5308, 1.0021],
         [1.9466, 0.1601, 0.8646, 1.0555, 1.1583],
         [0.4484, 0.5603, 1.0803, 1.4068, 1.5977],
         [0.6517, 0.9616, 0.7571, 1.4723, 0.9496]]),
 tensor([[1.1556, 1.3609, 1.1093, 1.5105, 0.4990],
         [1.0492, 1.2585, 0.1961, 0.5356, 1.4983],
         [0.8917, 1.0827, 1.8723, 0.3397, 0.8728],
         [0.9005, 0.6415, 1.1496, 0.4866, 0.1853],
         [1.8664, 0.7523, 1.2544, 0.5435, 1.5097]]))

## Test Layers

In [8]:
# Regular conv layer
conv = nn.Conv2d(128, 256, 1)
y1 = conv(rand1)
y1.shape

torch.Size([16, 256, 224, 224])

In [9]:
%%timeit
conv(rand1)

761 ms ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Oct layer that takes conv input, outputs TensorTuple
octconv = OctConv(128, 256, alpha=(0, 0.125), kernel_size=1, stride=1)
oct1 = octconv(rand1)
oct1.shape

(torch.Size([16, 224, 224, 224]), torch.Size([16, 32, 112, 112]))

In [11]:
%%timeit
octconv(rand1)

767 ms ± 6.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
convs = conv_layer(256, 512, (0, 0))
octs = conv_layer(256, 512, (0.125, 0.5))

In [13]:
%%timeit
convs(y1)

7.32 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
octs(oct1)

11.8 s ± 375 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test Full Models

I'm choosing weird image dimensions and alpha on purpose, to make sure that dimensions match up even if we're not at a power of 2.

In [15]:
device=torch.device('cpu')
model1 = octxresnet50(0).to(device)
model2 = octxresnet50(0.38).to(device)

In [16]:
rand_img = torch.rand((8, 3, 275, 275)).to(device)

In [17]:
%%timeit
model1(rand_img)

1.08 s ± 39.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
#The time takes twice as long!!! Haven't quite figured out why
model2(rand_img)

2.07 s ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
cut1 = nn.Sequential(*(list(model1.children())[:4]))
cut2 = nn.Sequential(*(list(model2.children())[:4]))

In [20]:
y1 = cut1(rand_img)
y2 = cut2(rand_img)
y1.shape, y2.shape

(torch.Size([8, 64, 69, 69]),
 (torch.Size([8, 40, 69, 69]), torch.Size([8, 24, 35, 35])))

In [21]:
%%timeit
cut1(rand_img)

138 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
cut2(rand_img)

168 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
